We will load the input data as dataframe:

In [21]:
import os
from sqlalchemy import create_engine
from tqdm import tqdm
import psycopg2
import json
import pandas as pd

with open('../credentials.json') as f:
    data = json.load(f)
    psql_config = {
    'dbname': data['db_name'],
    'user': data['db_user'],
    'password': data['db_pwd'],
    'host': data['db_host'],
    'port': 5432
}


def get_psql_connection():
    try:
        conn = psycopg2.connect(**psql_config)
    except Exception as e:
        print("Error connecting to the database:", e)
    return conn

In [30]:
engine = create_engine(f'postgresql://{psql_config["user"]}:{psql_config["password"]}@{psql_config["host"]}/{psql_config["dbname"]}')

df = pd.read_sql("SELECT * FROM m024.citi_bike_data LIMIT 1000;", engine)

df.head()

,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender,processed
0,2487,2018-06-20 10:11:52.580,2018-06-20 10:53:19.782,3349,Grand Army Plaza & Plaza St West,40.672968,-73.97088,3648,Flushing Ave & Vanderbilt Ave,40.697950,-73.970776,29165,Subscriber,1978,1,False
1,148,2018-06-20 12:23:14.847,2018-06-20 12:25:43.639,3349,Grand Army Plaza & Plaza St West,40.672968,-73.97088,3354,3 St & Prospect Park West,40.668132,-73.973638,32557,Subscriber,1973,1,False
2,384,2018-06-20 12:42:04.035,2018-06-20 12:48:28.717,3349,Grand Army Plaza & Plaza St West,40.672968,-73.97088,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,20569,Subscriber,1965,1,False
3,195,2018-06-20 13:07:46.286,2018-06-20 13:11:01.309,3349,Grand Army Plaza & Plaza St West,40.672968,-73.97088,3663,Sterling Pl & 7 Ave,40.677165,-73.973828,15720,Subscriber,1980,2,False
4,613,2018-06-20 13:16:43.348,2018-06-20 13:26:56.473,3349,Grand Army Plaza & Plaza St West,40.672968,-73.97088,3373,3 St & 3 Ave,40.675070,-73.987752,32058,Subscriber,1972,2,False


Now that we have the data, we will start processing it in bunch.
For each chunk, we will apply some validation and transform the source data into fact and dimension data

In [33]:
from math import radians, cos, sin, sqrt, atan2
from psycopg2.extras import execute_values
BATCH_SIZE = 100000

# Function to calculate distance using Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat, dlon = lat2 - lat1, lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

# Extract Data
def extract_data(offset):
    query = f"""
    SELECT * FROM m024.citi_bike_data 
    WHERE processed = FALSE 
    ORDER BY starttime 
    LIMIT {BATCH_SIZE} OFFSET {offset}
    """
    return pd.read_sql(query, engine)

# Load data into dimension tables
def load_dimension_data(df, table_name, cols, db_cols):
    df = df[cols].drop_duplicates()
    try:
        with get_psql_connection() as conn:
            with conn.cursor() as cur:
                statement = f"""
                INSERT INTO m024.p_{table_name} ({', '.join(db_cols)})
                VALUES %s 
                ON CONFLICT DO NOTHING;"""
                data_tuples = [tuple(row) for row in df.to_numpy()]
                    
                # Execute batch insert
                execute_values(cur, statement, data_tuples)
                
                # Commit changes
                conn.commit() 
                print(f"Data inserted/updated successfully in {table_name}")
    except Exception as e:
        print(f"Error in truncating citibike table - {e}")

In [ ]:


# Load dimension tables and return mapping IDs
def get_dimension_id(df, table_name, lookup_col, db_lookup_col,db_return_col):
    lookup_values = df[lookup_col].drop_duplicates().tolist()
    query = f"SELECT {db_lookup_col}, {db_return_col} FROM m024.p_{table_name} WHERE {db_lookup_col} IN :lookup_values"
    mapping = pd.read_sql(query, engine, params={"lookup_values": tuple(lookup_values)})
    return dict(zip(mapping[db_lookup_col], mapping[db_return_col]))



# Transform Data
def transform_data(df):
    #transform gender and load the dimension
    df.gender = df.gender.map(dict(zip([1, 2],['Male','Female']))).fillna('Unknown')
    load_dimension_data(df, 'gender_dimension', ['gender'],['gender_type'])
    df.usertype = df.usertype.apply(lambda x: x if x in ['Subscriber', 'Customer'] else 'Unknown')
    load_dimension_data(df, 'user_type_dimension', ['usertype'],['user_type'])
    df.birth_year = df.birth_year.apply(lambda x: x if x > 1940 and x <2013 else 0) # Assuming you need to be atleast 5 to ride the bike
    load_dimension_data(df, 'user_birthyear_dimension', ['birth_year'],['birth_year'])

    # Clean station names
    df['start_station_name'] = df['start_station_name'].str.strip().fillna('Unknown')
    df['end_station_name'] = df['end_station_name'].str.strip().fillna('Unknown')

    # Validate Latitude and Longitude
    df['start_station_latitude'] = df['start_station_latitude'].apply(
        lambda x: x if -90 <= x <= 90 else None
    )
    df['start_station_longitude'] = df['start_station_longitude'].apply(
        lambda x: x if -180 <= x <= 180 else None
    )
    df['end_station_latitude'] = df['end_station_latitude'].apply(
        lambda x: x if -90 <= x <= 90 else None
    )
    df['end_station_longitude'] = df['end_station_longitude'].apply(
        lambda x: x if -180 <= x <= 180 else None
    )

    # For missing latitude/longitude values
    df['start_station_latitude'].fillna('Unknown', inplace=True)
    df['start_station_longitude'].fillna('Unknown', inplace=True)
    df['end_station_latitude'].fillna('Unknown', inplace=True)
    df['end_station_longitude'].fillna('Unknown', inplace=True)
    load_dimension_data(df, 'station_dimension',
                         ['start_station_id', 'start_station_name', 'start_station_latitude', 'start_station_longitude'],
                           ['station_key', 'station_name', 'latitude', 'longitude'])
    load_dimension_data(df, 'station_dimension',
                         ['end_station_id', 'end_station_name', 'end_station_latitude', 'end_station_longitude'],
                           ['station_key', 'station_name', 'latitude', 'longitude'])
    
    # Fill missing values
    df['starttime'].fillna('1.1.1900', inplace=True)
    df['stoptime'].fillna('1.1.1900', inplace=True)

    # Convert to datetime format
    df['starttime_dt'] = pd.to_datetime(df['starttime'])
    df['stoptime_dt'] = pd.to_datetime(df['stoptime'])

    # Extract fields for start time
    df['start_date'] = df['starttime_dt'].dt.date
    df['start_year'] = df['starttime_dt'].dt.year
    df['start_month'] = df['starttime_dt'].dt.month
    df['start_day'] = df['starttime_dt'].dt.day
    df['start_hour'] = df['starttime_dt'].dt.hour

    # Extract fields for stop time
    df['stop_date'] = df['stoptime_dt'].dt.date
    df['stop_year'] = df['stoptime_dt'].dt.year
    df['stop_month'] = df['stoptime_dt'].dt.month
    df['stop_day'] = df['stoptime_dt'].dt.day
    df['stop_hour'] = df['stoptime_dt'].dt.hour

    load_dimension_data(df, 'time_dimension',
                         ['starttime', 'start_date', 'start_year', 'start_month', 'start_day', 'start_hour'],
                         ['time', 'date', 'year', 'month', 'day', 'hour'])
    
    load_dimension_data(df, 'time_dimension',
                         ['stoptime', 'stop_date', 'stop_year', 'stop_month', 'stop_day', 'stop_hour'],
                         ['time', 'date', 'year', 'month', 'day', 'hour'])
    
    load_dimension_data(df, 'bike_dimension', ['bikeid'], ['bike_id'])

    # Calculate trip distance
    df['distance'] = df.apply(lambda row: haversine(
        row['start_station_latitude'], row['start_station_longitude'], 
        row['end_station_latitude'], row['end_station_longitude']
    ), axis=1)

    # Get dimension table mappings
    station_map = get_dimension_id(df, 'station_dimension', 'start_station_id', 'station_key','station_id')
    end_station_map = get_dimension_id(df, 'station_dimension', 'end_station_id', 'station_key','station_id')
    time_map = get_dimension_id(df, 'time_dimension', 'starttime', 'time_id')
    stop_time_map = get_dimension_id(df, 'time_dimension', 'stoptime', 'time_id')
    user_type_map = get_dimension_id(df, 'user_type_dimension', 'usertype', 'user_type','user_type_id')
    gender_map = get_dimension_id(df, 'gender_dimension', 'gender', 'gender_type','gender_type_id')
    birth_year_map = get_dimension_id(df, 'user_birthyear_dimension', 'birth_year', 'birth_year', 'user_birthyear_id')

    # Map dimension table IDs
    df['start_time_id'] = df['time'].map(time_map)
    df['end_time_id'] = df['time'].map(stop_time_map)
    df['start_station_id'] = df['start_station_id'].map(station_map)
    df['end_station_id'] = df['end_station_id'].map(end_station_map)
    df['bike_id'] = df['bikeid']
    df['user_type_id'] = df['usertype'].map(user_type_map)
    df['gender_type_id'] = df['gender'].map(gender_map)
    df['user_birthyear_id'] = df['birth_year'].map(birth_year_map)

    return df[['tripduration', 'distance', 'start_time_id', 'end_time_id', 'start_station_id', 'end_station_id', 'bike_id', 'user_type_id', 'gender_type_id', 'user_birthyear_id']]

# Load fact table
def load_fact_table(df):
    df.to_sql('trip_fact', con=engine, if_exists='append', index=False, method='multi', chunksize=10000)

# Update processed records
def update_processed():
    with engine.begin() as conn:
        conn.execute(f"UPDATE m024.citi_bike_data SET processed = TRUE WHERE processed = FALSE")

# Process batch
def process_batch(offset):
    df = extract_data(offset)
    if not df.empty:
        df_transformed = transform_data(df)
        load_fact_table(df_transformed)
        update_processed()
        print(f"Processed batch {offset}-{offset+BATCH_SIZE}")

# Run ETL
def main():
    total_query = f"SELECT COUNT(*) FROM m024.citi_bike_data WHERE processed = FALSE"
    total_records = pd.read_sql(total_query, engine).iloc[0, 0]
    offsets = list(range(0, total_records, BATCH_SIZE))

    from multiprocessing import Pool
    with Pool(4) as pool:
        pool.map(process_batch, offsets)

if __name__ == "__main__":
    main()

